# [모듈 1.1] CodeCommit 리파지토리 생성

이 노트북은 CodeCommit 리파지토리를 생성하고, codecommit 폴더를 리파지토리에 Push 하는 노트북 입니다.

## 참고 자료
- [Quick guide to start using AWS CodeCommit via AWS CLI](https://dev.to/tiamatt/quick-guide-to-start-using-aws-codecommit-via-aws-cli-1jg5)
- 개발자 가이드
    - [Setup for HTTPS users using Git credentials](https://docs.aws.amazon.com/codecommit/latest/userguide/setting-up-gc.html)
    - [Create an AWS CodeCommit repository](https://docs.aws.amazon.com/codecommit/latest/userguide/how-to-create-repository.html#how-to-create-repository-cli)
    



# 1. 환경 확인

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [2]:
%%sh 
git --version

git version 2.37.1


In [3]:
%%sh
aws --version

aws-cli/1.24.10 Python/3.8.12 Linux/5.10.102-99.473.amzn2.x86_64 botocore/1.26.10


# 2. 파라미터 로딩

In [4]:
%store -r code_pipeline_train_config_json_path

from common_utils import load_json

code_pipeline_train_dict = load_json(code_pipeline_train_config_json_path)

import json
print (json.dumps(code_pipeline_train_dict, indent=2))

repository_name = code_pipeline_train_dict["train_code_repo_name"]

{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/codebuild-gsmoon",
  "project_prefix": "CodePipeline-Train-NCF",
  "region": "us-east-1",
  "account_id": "057716757052",
  "train_code_repo_name": "ncf-train",
  "code_build_project_name": "ncf-training-sm-pipeline",
  "bucket": "sagemaker-us-east-1-057716757052",
  "code_pipeline_name": "ncf-training-code-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "branch_name": "master"
}


In [5]:

source_code = 'codecommit'


# 3.리파지토리 생성

In [6]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit create-repository --repository-name $repository_name \
--repository-description "My demonstration repository" 

{
    "repositoryMetadata": {
        "accountId": "057716757052",
        "repositoryId": "61274fd1-a73c-456f-a8df-6e3d5dc0d2db",
        "repositoryName": "ncf-train",
        "repositoryDescription": "My demonstration repository",
        "lastModifiedDate": 1668931527.038,
        "creationDate": 1668931527.038,
        "cloneUrlHttp": "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train",
        "cloneUrlSsh": "ssh://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train",
        "Arn": "arn:aws:codecommit:us-east-1:057716757052:ncf-train"
    }
}


# 4. 리파지토리 로컬에 클로닝
### 아래를 위의 결과에서 카피해서 대체 해주세요.
#### clone_url =  "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/mlops-ncf"


In [7]:
clone_url =  "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train"
# clone_url = "<Clone URL>"

In [8]:
%%sh -s {clone_url} {repository_name}

clone_url=$1
repository_name=$2

rm -rf $repository_name

git clone $clone_url 

Cloning into 'ncf-train'...


# 5. 소스 코드 복사

In [9]:
%%sh -s {repository_name} {source_code}
repository_name=$1
source_code=$2

cp -r $source_code/* $repository_name
ls $repository_name

backup
codebuild-buildspec-221120.yml
codebuild-buildspec.yml
code_location.json
CONTRIBUTING.md
img
LICENSE
origin-sagemaker-pipelines-project.ipynb
pipelines
README.md
sagemaker-pipelines-project.ipynb
setup.cfg
setup.py
tests
tox.ini


# 6. 코드 파일을 리파지토리에 푸시

In [10]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
echo $PWD
git add .
git commit -m"Add new files"
git push origin master


/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/4_sm-train-codepipeline/ncf-train
[master (root-commit) ff7056c] Add new files
 Committer: EC2 Default User <ec2-user@ip-172-16-47-148.ec2.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 71 files changed, 7355 insertions(+)
 create mode 100644 CONTRIBUTING.md
 create mode 100644 LICENSE
 create mode 100644 README.md
 create mode 100644 backup/codebuild-buildspec-origin.yml
 create mode 100644 code_location.json
 create mode 100644 codebuild-buildspec-221120.yml
 create mode 100644 codebuild-buildspec.yml
 create mode 100644 img/pipeline-full.png
 create m

To https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train
 * [new branch]      master -> master
